In [6]:
from transformers import QuestionAnsweringPipeline, AutoAdapterModel, AutoModelWithHeads, AutoTokenizer, AutoConfig
from transformers.onnx import OnnxConfig, validate_model_outputs, export
from transformers.models.bert import BertOnnxConfig
from transformers.models.bart import BartOnnxConfig

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
from onnxruntime import InferenceSession
import onnxruntime

from onnx_opcounter import calculate_params

import os
import time
import torch
import numpy as np

from datasets import load_metric, load_dataset

from typing import Mapping, OrderedDict
from pathlib import Path
import random
import pandas as pd

In [7]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = AutoModelWithHeads.from_pretrained("facebook/bart-base")
adapter_name = model.load_adapter("AdapterHub/narrativeqa", source="hf", set_active=True)
model.set_active_adapters(adapter_name)

/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/models/bart/adapter_model.py:248: FutureWarning: This class has been renamed to `BartAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/models/bart/adapter_model.py:226: FutureWarning: This class has been renamed to `BartAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 4138.44it/s]


In [8]:
question, text = "What does Sara hate?", "Sara hates taxes. She loves vanilla ice cream."
prompt = text + "</s>" + question + "</s>"

encoding = tokenizer(prompt, return_tensors='pt', padding=False)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

answer = model.generate(input_ids, attention_mask=attention_mask, num_beams=4, max_length=128, early_stopping=True)
answer = tokenizer.decode(answer[0], skip_special_tokens=True)
answer


' Taxes \n'

In [9]:
logits = model(input_ids, attention_mask=attention_mask).logits[0]
print(logits.shape)
print(logits.argmax())

tokenizer.decode(logits[-1:, :].argmax(dim=1))[0]

torch.Size([20, 50265])
tensor(0)


'<'

In [10]:
cur_len = 0
max_length = 10
eos_token_id = (
    tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
)
features = encoding
input_ids = logits
generated_ids = []
unfinished_sequences = input_ids.new(input_ids.shape[0]).fill_(1)
scores = ()

In [11]:
# We need manual generation for the ONNX model (greedy generation)
while cur_len < max_length:
    input_data = features.copy()
    input_data["input_ids"] = torch.cat(
        (
            features["input_ids"],
            torch.tensor(generated_ids, dtype=int).unsqueeze(dim=0),
        ),
        dim=1,
    )
    input_data["attention_mask"] = torch.ones(input_data["input_ids"].shape, dtype=torch.int64)

    predictions = model(input_data["input_ids"], attention_mask=input_data["attention_mask"])

    next_token_logits = predictions["logits"][:, -1, :]
    scores += (next_token_logits,)

    # argmax
    next_tokens = torch.argmax(next_token_logits, dim=-1)
    # update generated ids, model inputs, and length for next step
    generated_ids.append(next_tokens[:, None].item())
    cur_len = cur_len + 1

    if eos_token_id is not None:
        unfinished_sequences = unfinished_sequences.mul((next_tokens != eos_token_id).long())
        # stop when each sentence is finished, or if we exceed the maximum length
    if unfinished_sequences.max() == 0:
        break


In [12]:
generated_ids

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
config = AutoConfig.from_pretrained("facebook/bart-base")
onnx_config = BartOnnxConfig.from_model_config(config, task="causal-lm")

onnx_path = Path("onnx/narrativeqabart/model.onnx")

onnx_inputs, onnx_outputs = export(tokenizer, model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/models/bart/modeling_bart.py:250: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/models/bart/modeling_bart.py:257: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/models/bart/modeling_bart.py:289: Tra

KeyboardInterrupt: 

In [14]:
onnx_inputs

['input_ids', 'attention_mask']

In [19]:
onnx_model = InferenceSession(
    str(onnx_path), providers=["CPUExecutionProvider"]
)

encoding = tokenizer(question, text, return_tensors='np')
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

outputs = onnx_model.run(input_feed=dict(encoding), output_names=None)

outputs

[array([[[30.675707  ,  3.679289  , 11.935048  , ...,  3.3267808 ,
           3.516271  , -3.3814769 ],
         [-8.085179  , -5.7296076 , -1.669905  , ..., -5.304813  ,
          -5.4988723 , -5.624103  ],
         [-5.5636964 , -4.7784514 ,  2.1397166 , ..., -4.5751605 ,
          -4.6781487 ,  0.81019855],
         ...,
         [ 0.41185176,  4.9883056 ,  1.7541847 , ...,  2.3238065 ,
           2.1625607 , -8.073326  ],
         [ 0.41218436,  4.988228  ,  1.7548296 , ...,  2.323756  ,
           2.1624923 , -8.073664  ],
         [ 0.41163564,  4.9882956 ,  1.7547405 , ...,  2.3238852 ,
           2.1626616 , -8.0732    ]]], dtype=float32),
 array([[[-0.03363782,  0.00642321, -0.01001072, ..., -0.00211524,
          -0.01134961,  0.02103707],
         [ 0.00783383,  0.17353505,  0.08948383, ...,  0.10017681,
          -0.02552552, -0.25290546],
         [-0.05093537,  0.21927737, -0.26953176, ..., -0.03408422,
           0.02267775, -0.14877988],
         ...,
         [ 0.21313